In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Withdrawal datasets

### BA

In [2]:
def transform_percentage(x):
    y = x.replace(',','.')
    return float(y.strip('%'))/100

In [3]:
df_BA = pd.read_csv('../data/raw_data/withdrawal_BA.csv', sep=';', encoding='utf-8')
df_BA['Isolated_city'] = df_BA['Isolated_city'].apply(transform_percentage)
df_a = df_BA.pivot(index='City Name', columns='Dt')['Isolated_city']
df_a = df_a.T
df_a.index = pd.to_datetime(df_a.index, format='%d/%m/%Y')
df_a = df_a.rename_axis(None)
df_a.columns = map(str.upper, df_a.columns)
df_a.index.name = 'date'
# df_a 

### SP

In [4]:
df_SP = pd.read_csv('../data/raw_data/withdrawal_SP.csv', sep='\t', encoding='utf-16', header=1)
df_b = df_SP.drop(['UF1','Código Município IBGE','População estimada (2020)'], axis=1).T
df_b.columns = df_b.iloc[0]
df_b = df_b.drop(df_b.index[0])
df_b.index = pd.to_datetime(df_b.index, format='%d/%m/%y')

for i in df_b.columns:
    df_b[i] = (df_b[i].str.replace('%',''))
    df_b[i] = df_b[i].astype(float)/100

df_b.columns = map(str.upper, df_b.columns)
df_b.index.name = 'date'
# df_b

In [5]:
df_withdrawal = pd.merge(df_a, df_b, on='date', how='inner')
df_withdrawal = df_withdrawal.T.reset_index().melt('index', var_name='date', value_name='withdrawal_SP')
df_withdrawal.rename(columns={'index': 'city_name'}, inplace=True)
# df_withdrawal

### States

In [6]:
oi = pd.read_csv('../data/raw_data/withdrawal_states.csv', sep=',', encoding='utf-8')
oi = oi.drop(['Dt-alias', 'AGG(avg_ponderada_br)-alias'], axis=1)
oi = oi.rename(columns={'Dt-value': 'data', 'AGG(avg_ponderada_br)-value': 'withdrawal', 'AGG(state_title)-alias': 'state'})
oi = oi.pivot(index='state', columns='data')['withdrawal'].T
oi.index = pd.to_datetime(oi.index, format='%Y/%m/%d')
oi = oi.rename_axis(None)
oi.columns = map(str.upper, oi.columns)

oi = oi.T.reset_index().melt('index', var_name='date', value_name='withdrawal_STATES')
oi.rename(columns={'index': 'STATE'}, inplace=True)

dates = df_withdrawal['date'].unique()

oi = oi[oi['date'].isin(list(dates))]
oi

,STATE,date,withdrawal_STATES
675,ACRE,2020-02-26,0.375934
676,ALAGOAS,2020-02-26,0.356466
677,AMAPÁ,2020-02-26,0.382863
678,AMAZONAS,2020-02-26,0.403251
679,BAHIA,2020-02-26,0.332213
...,...,...,...
10579,RORAIMA,2021-02-26,0.357533
10580,SANTA CATARINA,2021-02-26,0.290402
10581,SERGIPE,2021-02-26,0.314083
10582,SÃO PAULO,2021-02-26,0.290853


## Simple data cities

In [7]:
df_GDP = pd.read_csv('../data/raw_data/GDP.csv', sep=';', header=4)
df_GDP = df_GDP.iloc[:-12]
df_GDP = df_GDP.rename(columns={'1': 'city_code', 'Brasil': 'city_name', '7004141000': 'gdp'})
a = df_GDP['city_name'].str.split(' \(', expand=True)
df_GDP['state'] = a[1].str.replace('\)', '')
df_GDP['city_name'] = a[0].apply(str.upper)
df_GDP_gross = df_GDP
df_GDP = df_GDP[(df_GDP['state']=='BA') | (df_GDP['state']=='SP')]
df_GDP['city_code'] = df_GDP['city_code'].astype(int)
# df_GDP = df_GDP.set_index('city_code')
# df_GDP

/home/josebarretto/anaconda3/envs/study/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [8]:
df_pop = pd.read_csv('../data/raw_data/population.csv')
df_pop = df_pop.drop(columns=['Unnamed: 0', 'year'])
df_pop['city_code'] = df_pop['city_code'].astype(int)
# df_pop = df_pop.set_index('city_code')
# df_pop

In [9]:
df_san = pd.read_csv('../data/raw_data/sanitation.csv', sep=';', header=4)
df_san = df_san.rename(columns={
    'Cód.': 'city_code',
    'Total com algum serviço de saneamento básico': 'serv_sanitation',
    'Rede geral de distribuição de água': 'serv_water',
    'Rede coletora de esgoto': 'serv_sewage',
    'Manejo de resíduos sólidos': 'serv_garbage',
    'Manejo de águas pluviais': 'serv_rainwater'})
df_san = df_san.iloc[1:-12].drop(columns=['Brasil e Município', 'Total geral de municípios'])

df_san = df_san.astype(str)
for i in df_san.columns:
    df_san[i] = df_san[i].str.replace('-', '0')

df_san = df_san.astype(float)
# df_san

In [10]:
df_urb = pd.read_csv('../data/raw_data/urbanization.csv', sep=';', header=4)
df_urb = df_urb.rename(columns={'Cód.': 'city_code', 'Urbana': 'urbanization'})
df_urb = df_urb.iloc[1:-12].drop(columns=['Brasil e Município', 'Número de moradores'])
df_urb['urbanization'] = df_urb['urbanization'].str.replace(',','.')
df_urb['urbanization'] = pd.to_numeric(df_urb['urbanization'], downcast='float')
df_urb['city_code'] = df_urb['city_code'].astype(int)
# df_urb

In [11]:
df_work = pd.read_csv('../data/raw_data/work_type.csv', sep=';', header=4)
df_work = df_work.rename(columns={'Cód.': 'city_code'})
df_work = df_work.iloc[24:-12].drop(columns=['Brasil e Município', 'Posição na ocupação, subgrupo e categoria do emprego e contribuição para instituto de previdência oficial no trabalho principal'])
df_work['Total'] = df_work['Total'].str.replace(',','.')
df_work['Total'] = df_work['Total'].str.replace('-','0')
df_work['Total'] = pd.to_numeric(df_work['Total'], downcast='float')
df_work['city_code'] = df_work['city_code'].astype(int)
df_work = df_work.pivot(index='city_code', columns='Seção de atividade do trabalho principal')['Total']
# df_work

In [12]:
blue_collar = [
    'Transporte, armazenagem e correio',
    'Água, esgoto, atividades de gestão de resíduos e descontaminação',
    'Serviços domésticos',
    'Indústrias de transformação',
    'Indústrias extrativas',
    'Eletricidade e gás',
    'Construção',
    'Comércio; reparação de veículos automotores e motocicletas',
    'Agricultura, pecuária, produção florestal, pesca e aquicultura',
    'Alojamento e alimentação']
white_collar = [
    'Organismos internacionais e outras instituições extraterritoriais',
    'Outras atividades de serviços', 
    'Saúde humana e serviços sociais',
    'Educação',
    'Atividades administrativas e serviços complementares',
    'Atividades financeiras, de seguros e serviços relacionados',
    'Atividades imobiliárias', 'Atividades mal especificadas',
    'Atividades profissionais, científicas e técnicas',
    'Artes, cultura, esporte e recreação',
    'Administração pública, defesa e seguridade social']

In [13]:
df_work_spec = pd.DataFrame()
df_work_spec['white_collar'] = df_work[white_collar].sum(axis=1)
df_work_spec['blue_collar'] = df_work[blue_collar].sum(axis=1)
# df_work_spec

In [14]:
df_cities = pd.merge(df_GDP, df_pop, on='city_code', how='left')
df_cities = pd.merge(df_cities, df_san, on='city_code', how='left')
df_cities = pd.merge(df_cities, df_urb, on='city_code', how='left')
df_cities = pd.merge(df_cities, df_work_spec, on='city_code', how='left')

df_cities_static = df_cities
df_cities_static

,city_code,city_name,gdp,state,population,area,serv_sanitation,serv_water,serv_sewage,serv_garbage,serv_rainwater,urbanization,white_collar,blue_collar
0,2900108,ABAÍRA,59565.0,BA,8767,538.677,1.0,1.0,1.0,1.0,1.0,46.349998,23.010000,76.880005
1,2900207,ABARÉ,137847.0,BA,19814,1604.923,1.0,1.0,1.0,1.0,1.0,54.889999,19.929998,80.090004
2,2900306,ACAJUTIBA,133899.0,BA,15129,181.475,1.0,1.0,0.0,1.0,1.0,88.300003,20.170000,79.650002
3,2900355,ADUSTINA,112228.0,BA,16784,629.099,1.0,1.0,1.0,1.0,1.0,37.389999,14.860001,85.059998
4,2900405,ÁGUA FRIA,110270.0,BA,16901,742.775,1.0,1.0,0.0,1.0,1.0,39.720001,15.050000,84.810005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,3557006,VOTORANTIM,3093368.0,SP,121331,183.517,1.0,1.0,1.0,1.0,1.0,96.389999,27.780001,70.620003
1058,3557105,VOTUPORANGA,2928688.0,SP,93736,420.703,1.0,1.0,1.0,1.0,1.0,97.279999,32.400002,66.690002
1059,3557154,ZACARIAS,88517.0,SP,2684,319.056,1.0,1.0,1.0,1.0,1.0,79.849998,32.700001,67.070000
1060,3557204,CHAVANTES,207456.0,SP,12418,188.727,1.0,1.0,1.0,1.0,1.0,92.879997,24.029999,75.269997


## Variable in Time data Cities

In [15]:
df_tv = pd.read_csv('../data/raw_data/cable_television.csv')
df_tv = df_tv.drop(columns=['Unnamed: 0', 'city', 'state'])
df_tv = df_tv.rename(columns={'accesses': 'cable_tv_accesses'})
# df_tv

In [16]:
df_band = pd.read_csv('../data/raw_data/fixed_bandwidth.csv')
df_band = df_band.drop(columns=['Unnamed: 0', 'city', 'state'])
df_band = df_band.rename(columns={'accesses': 'fixed_bandwidth_accesses'})

# df_band

In [17]:
df_fixtel = pd.read_csv('../data/raw_data/fixed_telephony.csv')
df_fixtel = df_fixtel.drop(columns=['Unnamed: 0', 'city', 'state'])
df_fixtel['city_code'] = df_fixtel['city_code'].astype(int)
df_fixtel = df_fixtel.rename(columns={'accesses': 'fixed_telephony_accesses'})
# df_fixtel

In [18]:
df_mobtel = pd.read_csv('../data/raw_data/mobile_telephony.csv')
df_mobtel = df_mobtel.drop(columns=['Unnamed: 0', 'city', 'state'])
df_mobtel = df_mobtel.rename(columns={'accesses': 'mobile_telephony_accesses'})
df_mobtel_sum = df_mobtel.groupby(['city_code', 'date', 'technology']).sum()
df_mobtel_sum.reset_index(inplace=True)
df_mobtel_sum = df_mobtel_sum.pivot(index=['city_code', 'date'], columns='technology')['mobile_telephony_accesses']
df_mobtel_sum.reset_index(inplace=True)

df_mobtel_sum = df_mobtel_sum.rename(columns={'2G': '2G_mobile_accesses',
                                              '3G': '3G_mobile_accesses',
                                              '4G': '4G_mobile_accesses'})
df_mobtel = df_mobtel_sum
# df_mobtel
# df_mobtel[df_mobtel['city_code']==3550308] #SP

In [19]:
df_time_cities = pd.merge(df_tv, df_band, on=['city_code', 'date'], how='left')
df_time_cities = pd.merge(df_time_cities, df_fixtel, on=['city_code', 'date'], how='left')
df_time_cities = pd.merge(df_time_cities, df_mobtel, on=['city_code', 'date'], how='left')
df_time_cities['date'] = pd.to_datetime(df_time_cities['date'])
df_time_cities.set_index('date', inplace=True)
# df_time_cities

In [20]:
cities_num = list(df_cities['city_code'].unique())

df_cities_resampled = pd.DataFrame()
for city in cities_num:
    df_ = df_time_cities[df_time_cities['city_code']==city]
    df_cities_resampled =  pd.concat([df_cities_resampled, df_.resample('1D').pad()])  

df_cities_resampled = df_cities_resampled[df_cities_resampled.index.isin(list(dates))]
df_cities_resampled = pd.merge(df_cities[['city_code', 'city_name']], df_cities_resampled.reset_index(), on=['city_code'], how='inner')
# df_cities_resampled

In [21]:
df_cities_final = pd.merge(df_withdrawal, df_cities_resampled, on=['city_name', 'date'], how='inner')
df_cities_final.sort_values(['city_code', 'date'], inplace=True)
df_cities_final

,city_name,date,withdrawal_SP,city_code,cable_tv_accesses,fixed_bandwidth_accesses,fixed_telephony_accesses,2G_mobile_accesses,3G_mobile_accesses,4G_mobile_accesses
165689,ABAÍRA,2020-02-26,0.333,2900108,946,217,744.0,662.0,724.0,2580.0
172917,ABAÍRA,2020-02-27,0.255,2900108,946,217,744.0,662.0,724.0,2580.0
179589,ABAÍRA,2020-02-28,0.207,2900108,946,217,744.0,662.0,724.0,2580.0
186261,ABAÍRA,2020-02-29,0.186,2900108,946,217,744.0,662.0,724.0,2580.0
1113,ABAÍRA,2020-03-01,0.333,2900108,987,218,735.0,638.0,718.0,2573.0
...,...,...,...,...,...,...,...,...,...,...
139555,VOTUPORANGA,2021-02-22,0.390,3557105,3004,8405,13934.0,6518.0,9833.0,83850.0
146227,VOTUPORANGA,2021-02-23,0.390,3557105,3004,8405,13934.0,6518.0,9833.0,83850.0
152899,VOTUPORANGA,2021-02-24,0.380,3557105,3004,8405,13934.0,6518.0,9833.0,83850.0
159571,VOTUPORANGA,2021-02-25,0.400,3557105,3004,8405,13934.0,6518.0,9833.0,83850.0


## COVID data

In [22]:
df_covid = pd.read_csv('../data/raw_data/COVID.csv.zip')
df_covid = df_covid.drop(columns=['Unnamed: 0'])
df_covid = df_covid.rename(columns={'city_ibge_code': 'city_code',
                                   'city': 'city_name',
                                   'confirmed': 'confirmed_acc',
                                   'deaths': 'deaths_acc'})

df_covid['city_name'] = df_covid['city_name'].str.upper()
df_covid = df_covid[((df_covid['state']=='BA') | (df_covid['state']=='SP'))]
df_covid['date'] = pd.to_datetime(df_covid['date'])
df_covid = df_covid[~df_covid['city_code'].isna()]
df_covid['city_code'] = df_covid['city_code'].astype(int)

a = pd.DataFrame()
b = pd.DataFrame()

for i in list(df_covid['city_code'].unique()):
    a = df_covid[df_covid['city_code']==i]
    a = a.sort_values(by=['date'], ascending=True)
    a[['confirmed_day', 'deaths_day']] = a[['confirmed_acc', 'deaths_acc']].diff()
    
    b = pd.concat([a,b])
b.loc[b['deaths_day']<0, 'deaths_day']=0
b.loc[b['confirmed_day']<0, 'confirmed_day']=0

df_covid = b

df_covid[['deaths_day', 'confirmed_day']] = df_covid[['deaths_day', 'confirmed_day']].fillna(0)
df_covid = df_covid[df_covid['date'].isin(list(dates))]
df_covid = df_covid[df_covid['city_code'].isin(list(cities_num))]
df_covid.sort_values(['city_code', 'date'], inplace=True)
df_covid.reset_index(drop=True, inplace=True)
df_covid

,date,state,city_name,confirmed_acc,deaths_acc,city_code,death_rate,confirmed_day,deaths_day
0,2020-04-13,BA,ABAÍRA,1,0,2900108,0.0000,0.0,0.0
1,2020-04-14,BA,ABAÍRA,1,0,2900108,0.0000,0.0,0.0
2,2020-04-15,BA,ABAÍRA,1,0,2900108,0.0000,0.0,0.0
3,2020-04-16,BA,ABAÍRA,1,0,2900108,0.0000,0.0,0.0
4,2020-04-17,BA,ABAÍRA,1,0,2900108,0.0000,0.0,0.0
...,...,...,...,...,...,...,...,...,...
304302,2021-02-22,SP,ESTIVA GERBI,319,11,3557303,0.0345,0.0,0.0
304303,2021-02-23,SP,ESTIVA GERBI,329,11,3557303,0.0334,10.0,0.0
304304,2021-02-24,SP,ESTIVA GERBI,332,11,3557303,0.0331,3.0,0.0
304305,2021-02-25,SP,ESTIVA GERBI,338,12,3557303,0.0355,6.0,1.0


In [32]:
df_final = pd.merge(df_cities_final, df_covid, on=['city_code', 'date'], how='inner')
df_final.drop(columns=['city_name_y'], inplace=True)
df_final.rename(columns={'city_name_x': 'city_name', 'withdrawal_SP': 'withdrawal'}, inplace=True)
df_final = df_final[[
    'city_name', 'city_code', 'date', 'state',
    'withdrawal', 'cable_tv_accesses', 'fixed_bandwidth_accesses', 'fixed_telephony_accesses', 
    '2G_mobile_accesses', '3G_mobile_accesses', '4G_mobile_accesses', 'confirmed_acc',
    'deaths_acc', 'death_rate', 'confirmed_day', 'deaths_day'
]]
df_final

,city_name,city_code,date,state,withdrawal,cable_tv_accesses,fixed_bandwidth_accesses,fixed_telephony_accesses,2G_mobile_accesses,3G_mobile_accesses,4G_mobile_accesses,confirmed_acc,deaths_acc,death_rate,confirmed_day,deaths_day
0,ABAÍRA,2900108,2020-04-13,BA,0.571,979,220,735.0,580.0,661.0,2354.0,1,0,0.0000,0.0,0.0
1,ABAÍRA,2900108,2020-04-14,BA,0.556,979,220,735.0,580.0,661.0,2354.0,1,0,0.0000,0.0,0.0
2,ABAÍRA,2900108,2020-04-15,BA,0.596,979,220,735.0,580.0,661.0,2354.0,1,0,0.0000,0.0,0.0
3,ABAÍRA,2900108,2020-04-16,BA,0.588,979,220,735.0,580.0,661.0,2354.0,1,0,0.0000,0.0,0.0
4,ABAÍRA,2900108,2020-04-17,BA,0.500,979,220,735.0,580.0,661.0,2354.0,1,0,0.0000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164488,VOTUPORANGA,3557105,2021-02-22,SP,0.390,3004,8405,13934.0,6518.0,9833.0,83850.0,7699,143,0.0186,6.0,0.0
164489,VOTUPORANGA,3557105,2021-02-23,SP,0.390,3004,8405,13934.0,6518.0,9833.0,83850.0,7720,144,0.0187,21.0,1.0
164490,VOTUPORANGA,3557105,2021-02-24,SP,0.380,3004,8405,13934.0,6518.0,9833.0,83850.0,7832,145,0.0185,112.0,1.0
164491,VOTUPORANGA,3557105,2021-02-25,SP,0.400,3004,8405,13934.0,6518.0,9833.0,83850.0,7859,146,0.0186,27.0,1.0


In [33]:
df_cities_static.to_csv('../data/proc_data/cities_static.csv.zip')
df_final.to_csv('../data/proc_data/cities_temp.csv.zip')
oi.to_csv('../data/proc_data/states_withdrawal.csv.zip')